## Import libraries and define functions

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def tidy(df):
    
    #### Fix usual issues with all strings
    
    # Capitalise headers
    df.columns = df.columns.astype(str).str.upper()
    
    # Capitalise columns
    df = df.map(lambda x: x.upper() if type(x) is str else x)

    # Strip whitespace
    df = df.map(lambda x: x.strip() if type(x) is str else x)

    # Remove parenthesis
    df = df.map(lambda x: x.replace('(', '') if type(x) is str else x)
    df = df.map(lambda x: x.replace(')', '') if type(x) is str else x)
    
    # Remove dashes
    df = df.map(lambda x: x.replace('-', '') if type(x) is str else x)
    
    # Remove full stops
    df = df.map(lambda x: x.replace('.', '') if type(x) is str else x)
    
    # Remove commas
    df = df.map(lambda x: x.replace(',', '') if type(x) is str else x)
    
    # Remove linebreaks
    df = df.map(lambda x: x.replace('\n', '') if type(x) is str else x)

    # Replace annoying substrings
    df = df.map(lambda x: x.replace(' AND ', ' & ') if type(x) is str else x)
    df = df.map(lambda x: x.replace(' – ', ' - ') if type(x) is str else x)
    df = df.map(lambda x: x.replace(' / ', '/') if type(x) is str else x)
    df = df.map(lambda x: x.replace('/ ', '/') if type(x) is str else x)
    df = df.map(lambda x: x.replace(' /', '/') if type(x) is str else x)
    
    # Strip whitespace again
    df = df.map(lambda x: x.strip() if type(x) is str else x)
    
    # Drop rows with minimum number of 2 non-null values
    df = df.dropna(thresh=2)
    
    return df

## Read in Mallaig data

In [ ]:
# Add in Mallaig High School data from previous FIO request
dfm = pd.read_excel('./data/FS560267659 budget and spend data.xlsx', 
                                sheet_name='10416')
    
dfm = tidy(dfm)
    
# Add SCHOOL and DEPT column
dfm['SCHOOL'] = 'MALLAIG HIGH SCHOOL'
dfm['DEPT'] = 'XX'
dfm['CATEGORY'] = 'XX'

# Rename Desciption column
dfm = dfm.rename(columns={'DESCR': 'DESCRIPTION'})

# Update a few names
dfm = dfm.map(lambda x: x.replace('TEACHERS SUPPLY', 'SUPPLY') if type(x) is str else x)
dfm = dfm.map(lambda x: x.replace('TEACHER SUPPLY', 'SUPPLY') if type(x) is str else x)

# Remove School name from desciption column
ls = ['MALLAIG HIGH SCHOOL', 'MALLAIG SECONDARY SCHOOL', 'MALLAIG SECONDARY',
        'MALLAIG SEC SCHOOL', 'MALLAIG SEC', ' PS ', 
        'MALLAIG', 'LADY LOVAT', 'ARISAIG', 'INVERIE']

for l in ls:
    dfm['DESCRIPTION'] = dfm['DESCRIPTION'].str.replace(l, '')

# Join together some awkward categories
dfm['DESCRIPTION'] = dfm['DESCRIPTION'].str.replace('LRN SUP', 'LRNSUP')
dfm['DESCRIPTION'] = dfm['DESCRIPTION'].str.replace('MOD LANG', 'MODLANG')
dfm['DESCRIPTION'] = dfm['DESCRIPTION'].str.replace('MOD STD', 'MODSTD')
dfm['DESCRIPTION'] = dfm['DESCRIPTION'].str.replace('MUSIC & DRAMA', 'MUSIC&DRAMA')
dfm['DESCRIPTION'] = dfm['DESCRIPTION'].str.replace('FIRST AID', 'FIRSTAID')
dfm['DESCRIPTION'] = dfm['DESCRIPTION'].str.replace('FOOD FOR THOUGHT', 'FOODFORTHOUGHT')
dfm['DESCRIPTION'] = dfm['DESCRIPTION'].str.replace('ADD FUNDS', 'ADDFUNDS')

# Drop rows with non-null values
dfm = dfm.dropna()

# Strip whitespace
dfm = dfm.map(lambda x: x.strip() if type(x) is str else x)

# Set Category column to first name of DESCRIPTION  column
dfm['SCHOOL CODE'] = dfm['DESCRIPTION'].str.split(n=1).str[0]

# Remove first word from DESCRIPTION column
dfm['DESCRIPTION'] = dfm['DESCRIPTION'].str.split(n=1).str[1]

# Create list of values
val = ['2324 ACTUALS', '2223 ACTUALS', '2122 ACTUALS', '2021 ACTUALS',
       '1920 ACTUALS', '1819 ACTUALS', '1718 ACTUALS', '1617 ACTUALS', '1516 ACTUALS']

# Convert into long format
ldfm = pd.melt(dfm, id_vars=['EXP HEAD', 'DESCRIPTION', 'SCHOOL', 'SCHOOL CODE',
                                 'CATEGORY', 'DEPT'], value_vars=val,
             var_name='VARIABLE', value_name='VALUE')

# Split variable column into year and type
ldfm[['YEAR', 'TYPE']] = ldfm['VARIABLE'].str.split(' ', n=1, expand=True)

# Drop old variable column
ldfm = ldfm.drop('VARIABLE', axis=1)

# Drop values equal to 0
ldfm = ldfm[ldfm['VALUE'] != 0]

# Reorder columns
ldfm = ldfm[['EXP HEAD', 'DEPT', 'SCHOOL', 'CATEGORY', 'SCHOOL CODE', 'DESCRIPTION', 'YEAR', 'VALUE']]

## Read in main data

In [ ]:
## Read in main source file

df = pd.read_excel('./data/FOI FS-Case-625356341 - School spend.xlsx',
                       sheet_name='Monitoring Reports',
                          skiprows = 7)

In [ ]:
# df.head()
# dfm.head()

In [ ]:
# Copy df
fdf = df.copy()

# Tidy df
fdf = tidy(fdf)

In [ ]:
# Rename Desciption column
fdf = fdf.rename(columns={'CC DESCRIPTION': 'SCHOOL CODE'})
fdf = fdf.rename(columns={'CC NAME': 'SCHOOL'})
fdf = fdf.rename(columns={'EH DESCRIPTION': 'DESCRIPTION'})

# Update a few names
fdf = fdf.map(lambda x: x.replace('TEACHERS SUPPLY', 'SUPPLY') if type(x) is str else x)
fdf = fdf.map(lambda x: x.replace('TEACHER SUPPLY', 'SUPPLY') if type(x) is str else x)

# Join together some awkward categories
fdf['DESCRIPTION'] = fdf['DESCRIPTION'].str.replace('LRN SUP', 'LRNSUP')
fdf['DESCRIPTION'] = fdf['DESCRIPTION'].str.replace('MOD LANG', 'MODLANG')
fdf['DESCRIPTION'] = fdf['DESCRIPTION'].str.replace('MOD STD', 'MODSTD')
fdf['DESCRIPTION'] = fdf['DESCRIPTION'].str.replace('MUSIC & DRAMA', 'MUSIC&DRAMA')
fdf['DESCRIPTION'] = fdf['DESCRIPTION'].str.replace('FIRST AID', 'FIRSTAID')
fdf['DESCRIPTION'] = fdf['DESCRIPTION'].str.replace('FOOD FOR THOUGHT', 'FOODFORTHOUGHT')
fdf['DESCRIPTION'] = fdf['DESCRIPTION'].str.replace('ADD FUNDS', 'ADDFUNDS')
fdf['DESCRIPTION'] = fdf['DESCRIPTION'].str.replace('ART & CRAFT', 'ART&CRAFT')
fdf['DESCRIPTION'] = fdf['DESCRIPTION'].str.replace('MEALS & HOSPITALITY', 'MEALS&HOSPITALITY')
fdf['DESCRIPTION'] = fdf['DESCRIPTION'].str.replace('RPRS & MAINT', 'RPRS&MAINT')
fdf['DESCRIPTION'] = fdf['DESCRIPTION'].str.replace('CONFERENCE & SEMINAR', 'CONFERENCE&SEMINAR')

# Drop rows with non-null values
fdf = fdf.dropna()

# Strip whitespace
fdf = fdf.map(lambda x: x.strip() if type(x) is str else x)

# Set Category column to first name of DESCRIPTIONiption column
fdf['CATEGORY'] = fdf['DESCRIPTION'].str.split(n=1).str[0]

# Remove first word from DESCRIPTIONiption column
fdf['DESCRIPTION'] = fdf['DESCRIPTION'].str.split(n=1).str[1]

fdf.head()

In [ ]:
# Remove School name from school code column
ls1 = fdf['SCHOOL'].unique()

for l in ls1:
    fdf['SCHOOL CODE'] = fdf['SCHOOL CODE'].str.replace(l, '')
    
# Remove shortened School name from school code column
ls2 = ['CHARLESTON AC', 'CULLODEN ACAD',
       'GLEN URQUHART HIGH', 'GLEN URQUHART HS', 'GLEN URQUHART HS', 'GLEN URQUHART','INVERNESS HIGH',
       'INVERNESS ROYAL AC', 'INVNESS ROYAL AC', 'INVERNESS ROYAL', 'KILCHUIMEN AC',
       'MILLBURN ACAD', 'MILBURN ACADEMY', 'MILLBURN', 'MILBURN',
       'KINLOCHLEVEN HIGH', 'KINLOCHLEVEN HS',
       'LOCHABER HIGH', 'ALNESS ACADEMY', 'DINGWALL ACAD',
       'FORTROSE ACAD', 'GAIRLOCH HIGH', 'INVERGORDON ACAD', 'INVERGORDON AC',
       'TAIN ROYAL ACAD', 'ULLAPOOL HIGH', 'DORNOCH ACAD', 'DORNOCH AC',
       'FARR SECONDARY', 'FARR SEC SCHOOL', 'FARR HIGH SCHOOL', 'GOLSPIE',
       'KINLOCHBERVIE HIGH', 'KINLOCHBERVIE HS', 'KINLOCHLEVEN H', 'KINLCHBERVIE', "K'BERVIE", 'KLB HIGH',
       'THURSO', 'WICK', 'PORTREE HIGH', 'PORTREE', 'PLOCKTON HIGH',
       'GRANTOWN GRAMMR', 'GRANTOWN GRAMM', 'GRANTOWN', 'KINGUSSIE HIGH', 'KINGUSSIE HS',
       'NAIRN ACAD', 'ARDNAMURCHAN SEC', 'ARDNAMURCHAN SS', 'ARDNAMURCHAN HIGH']

for l in ls2:
    fdf['SCHOOL CODE'] = fdf['SCHOOL CODE'].str.replace(l, '')
    
# Remove loose ends from school code column
ls3 = ['HIGH SCHOOL', 'HS ', 'INVERNESS', 'AR ', 'AD ']

for l in ls3:
    fdf['SCHOOL CODE'] = fdf['SCHOOL CODE'].str.replace(l, '')
     
# Tidy up again (mainly to remove whitespaces)
fdf = tidy(fdf)

# Correct typos (something using regex to select full string only)
fdf['SCHOOL CODE'] = fdf['SCHOOL CODE'].str.replace(r'^GOV INITIATIVE$', 'GOV INITIATIVES', regex=True)
fdf['SCHOOL CODE'] = fdf['SCHOOL CODE'].str.replace(r'^GOV INTIVE$', 'GOV INITIATIVES', regex=True)

fdf['SCHOOL CODE'] = fdf['SCHOOL CODE'].str.replace(r'^INNO FUND$', 'INNOVATION FUNDING', regex=True)
fdf['SCHOOL CODE'] = fdf['SCHOOL CODE'].str.replace(r'^INNO FUNDING$', 'INNOVATION FUNDING', regex=True)
fdf['SCHOOL CODE'] = fdf['SCHOOL CODE'].str.replace(r'^INNOVATION FUND$', 'INNOVATION FUNDING', regex=True)

fdf['SCHOOL CODE'] = fdf['SCHOOL CODE'].str.replace('ASG STEM FUNDING', 'STEM FUNDING')
fdf['SCHOOL CODE'] = fdf['SCHOOL CODE'].str.replace('TRA STEM FUNDING', 'STEM FUNDING')

fdf['SCHOOL CODE'] = fdf['SCHOOL CODE'].str.replace(r'^ACCESS TO EDU$', 'ACCESS TO EDUC', regex=True)

In [ ]:
fdf.head()

## Convert main data into long format

In [ ]:
# Convert into long format
ldf = pd.melt(fdf, id_vars=['EXP HEAD', 'DEPT', 'DESCRIPTION', 'SCHOOL', 'YEAR', 'CATEGORY', 'SCHOOL CODE'],
            value_vars='ACTUALS YTD',
                var_name='VARIABLE', value_name='VALUE')

# Drop old variable column
ldf = ldf.drop('VARIABLE', axis=1)

# Drop values equal to 0
ldf = ldf[ldf['VALUE'] != 0]

# Reorder columns
ldf = ldf[['EXP HEAD', 'DEPT', 'SCHOOL', 'CATEGORY', 'SCHOOL CODE', 'DESCRIPTION', 'YEAR', 'VALUE']]

In [ ]:
# ldf.head()
# ldfm.head()

## Join budget data together

In [ ]:
# Concat main source data with Mallaig data
tdf = pd.concat([ldf, ldfm])

In [ ]:
tdf.head()

In [ ]:
tdf.dtypes

## Read in school stats data (no output yet)

In [ ]:
# Create empty list to store all df
frames = []

In [ ]:
# Read in data
dfs = pd.read_excel('./data/School+level+summary+statistics+2023+v2.xlsx', 
                                sheet_name='2023 School Level Statistics',
                                   skiprows=1,
                                       na_values='c')

# Tidy data
dfs = tidy(dfs)

# Remove (%) from column headings
dfs.columns = dfs.columns.str.replace('(%)', '')

# Create list of suffixes to remove
ls = ['[NOTE 9]', '[NOTE 8]', '[NOTE 7]', '[NOTE 6]', '[NOTE 5]', '[NOTE 4]',
      '[NOTE 3]', '[NOTE 2]', '[NOTE 1]']

# Loop through suffix list
for i in ls:

    # Remove suffix
    dfs.columns = dfs.columns.str.removesuffix(i)
    
    # Strip white space
    dfs.columns = dfs.columns.str.strip()

# Rename column headers
dfs = dfs.rename(columns={'TEACHERS (FULL TIME EQUIVALENT)': 'TEACHERS'})
dfs = dfs.rename(columns={'PUPILS WITH AN ADDITIONAL SUPPORT NEED RECORDED': 'ASN PUPILS'})
dfs = dfs.rename(columns={'SCHOOL NAME': 'SCHOOL'})

# Select Highland Secondary schools
dfs = dfs.loc[dfs['LOCAL AUTHORITY'] == 'HIGHLAND']
dfs = dfs.loc[dfs['SCHOOL TYPE'] == 'SECONDARY']

# Select subset of columns
dfs = dfs[['SCHOOL', 'TEACHERS', 'PUPIL ROLL', 'ASN PUPILS', 'ATTENDANCE RATE']]

# Add year
dfs['YEAR'] = '2223'

# Append df to list
frames.append(dfs)

In [ ]:
# Read in data
dfs = pd.read_excel('./data/School+level+summary+statistics+2022+V4.xlsx', 
                                sheet_name='2022 School Level Statistics',
                                   skiprows=1,
                                       na_values='c')

# Tidy data
dfs = tidy(dfs)

# Remove (%) from column headings
dfs.columns = dfs.columns.str.replace('(%)', '')

# Create list of suffixes to remove
ls = ['[NOTE 9]', '[NOTE 8]', '[NOTE 7]', '[NOTE 6]', '[NOTE 5]', '[NOTE 4]',
      '[NOTE 3]', '[NOTE 2]', '[NOTE 1]']

# Loop through suffix list
for i in ls:

    # Remove suffix
    dfs.columns = dfs.columns.str.removesuffix(i)
    
    # Strip white space
    dfs.columns = dfs.columns.str.strip()

# Rename column headers
dfs = dfs.rename(columns={'TEACHERS (FULL TIME EQUIVALENT)': 'TEACHERS'})
dfs = dfs.rename(columns={'PUPILS WITH AN ADDITIONAL SUPPORT NEED RECORDED': 'ASN PUPILS'})
dfs = dfs.rename(columns={'SCHOOL NAME': 'SCHOOL'})

# Select Highland Secondary schools
dfs = dfs.loc[dfs['LOCAL AUTHORITY'] == 'HIGHLAND']
dfs = dfs.loc[dfs['SCHOOL TYPE'] == 'SECONDARY']

# Select subset of columns
dfs = dfs[['SCHOOL', 'TEACHERS', 'PUPIL ROLL', 'ASN PUPILS', 'ATTENDANCE RATE']]

# Add year
dfs['YEAR'] = '2122'

# Append df to list
frames.append(dfs)

In [ ]:
# Read in data
dfs = pd.read_excel('./data/School+level+summary+statistics+2021.xlsx', 
                                sheet_name='2021 School Level Statistics',
                                   skiprows=1,
                                       na_values='c')

# Tidy data
dfs = tidy(dfs)

# Remove (%) from column headings
dfs.columns = dfs.columns.str.replace('(%)', '')

# Create list of suffixes to remove
ls = ['[NOTE 9]', '[NOTE 8]', '[NOTE 7]', '[NOTE 6]', '[NOTE 5]', '[NOTE 4]',
      '[NOTE 3]', '[NOTE 2]', '[NOTE 1]']

# Loop through suffix list
for i in ls:

    # Remove suffix
    dfs.columns = dfs.columns.str.removesuffix(i)
    
    # Strip white space
    dfs.columns = dfs.columns.str.strip()

# Rename column headers
dfs = dfs.rename(columns={'TEACHERS (FULL TIME EQUIVALENT)': 'TEACHERS'})
dfs = dfs.rename(columns={'PUPILS WITH AN ADDITIONAL SUPPORT NEED RECORDED': 'ASN PUPILS'})
dfs = dfs.rename(columns={'SCHOOL NAME': 'SCHOOL'})

# Select Highland Secondary schools
dfs = dfs.loc[dfs['LOCAL AUTHORITY'] == 'HIGHLAND']
dfs = dfs.loc[dfs['SCHOOL TYPE'] == 'SECONDARY']

# Select subset of columns
dfs = dfs[['SCHOOL', 'TEACHERS', 'PUPIL ROLL', 'ASN PUPILS', 'ATTENDANCE RATE']]

# Add year
dfs['YEAR'] = '2021'

# Append df to list
frames.append(dfs)

In [ ]:
# Read in data
dfs = pd.read_excel('./data/School+level+summary+statistics+2020.xlsx', 
                                sheet_name='2020',
                                   skiprows=1,
                                       na_values='c')

# Tidy data
dfs = tidy(dfs)

# Remove (%) from column headings
dfs.columns = dfs.columns.str.replace('(%)', '')

# Create list of suffixes to remove
ls = ['(9)', '(8)', '(7)', '(6)', '(5)', '(4)', '(3)', '(2)', '(1)']

# Loop through suffix list
for i in ls:

    # Remove suffix
    dfs.columns = dfs.columns.str.removesuffix(i)
    
    # Strip white space
    dfs.columns = dfs.columns.str.strip()

# Rename column headers
dfs = dfs.rename(columns={'FTE TEACHERS': 'TEACHERS'})
dfs = dfs.rename(columns={'PUPILS WITH AN ADDITIONAL SUPPORT NEED RECORDED': 'ASN PUPILS'})
dfs = dfs.rename(columns={'SCHOOL NAME': 'SCHOOL'})

# Select Highland Secondary schools
dfs = dfs.loc[dfs['LOCAL AUTHORITY'] == 'HIGHLAND']
dfs = dfs.loc[dfs['SCHOOL TYPE'] == 'SECONDARY']

# Select subset of columns
dfs = dfs[['SCHOOL', 'TEACHERS', 'PUPIL ROLL', 'ASN PUPILS', 'ATTENDANCE RATE']]

# Add year
dfs['YEAR'] = '1920'

# Append df to list
frames.append(dfs)

In [ ]:
# Read in data
dfs = pd.read_excel('./data/School+level+summary+statistics+2019.xlsx', 
                                sheet_name='2019',
                                   skiprows=1,
                                       na_values='c')

# Tidy data
dfs = tidy(dfs)

# Remove (%) from colu\mn headings
dfs.columns = dfs.columns.str.replace('(%)', '')

# Create list of suffixes to remove
ls = ['(9)', '(8)', '(7)', '(6)', '(5)', '(4)', '(3)', '(2)', '(1)']

# Loop through suffix list
for i in ls:

    # Remove suffix
    dfs.columns = dfs.columns.str.removesuffix(i)
    
    # Strip white space
    dfs.columns = dfs.columns.str.strip()

# Rename column headers
dfs = dfs.rename(columns={'FTE TEACHERS': 'TEACHERS'})
dfs = dfs.rename(columns={'PUPILS WITH AN ADDITIONAL SUPPORT NEED RECORDED': 'ASN PUPILS'})
dfs = dfs.rename(columns={'SCHOOL NAME': 'SCHOOL'})

# Select Highland Secondary schools
dfs = dfs.loc[dfs['LOCAL AUTHORITY'] == 'HIGHLAND']
dfs = dfs.loc[dfs['SCHOOL TYPE'] == 'SECONDARY']

# Select subset of columns
dfs = dfs[['SCHOOL', 'TEACHERS', 'PUPIL ROLL', 'ASN PUPILS', 'ATTENDANCE RATE']]

# Add year
dfs['YEAR'] = '1819'

# Append df to list
frames.append(dfs)

In [ ]:
# Read in data
dfs = pd.read_excel('./data/School+level+summary+statistics+2019.xlsx', 
                                sheet_name='2019',
                                   skiprows=1,
                                       na_values='c')

# Tidy data
dfs = tidy(dfs)

# Remove (%) from colu\mn headings
dfs.columns = dfs.columns.str.replace('(%)', '')

# Create list of suffixes to remove
ls = ['(9)', '(8)', '(7)', '(6)', '(5)', '(4)', '(3)', '(2)', '(1)']

# Loop through suffix list
for i in ls:

    # Remove suffix
    dfs.columns = dfs.columns.str.removesuffix(i)
    
    # Strip white space
    dfs.columns = dfs.columns.str.strip()

# Rename column headers
dfs = dfs.rename(columns={'FTE TEACHERS': 'TEACHERS'})
dfs = dfs.rename(columns={'PUPILS WITH AN ADDITIONAL SUPPORT NEED RECORDED': 'ASN PUPILS'})
dfs = dfs.rename(columns={'SCHOOL NAME': 'SCHOOL'})

# Select Highland Secondary schools
dfs = dfs.loc[dfs['LOCAL AUTHORITY'] == 'HIGHLAND']
dfs = dfs.loc[dfs['SCHOOL TYPE'] == 'SECONDARY']

# Select subset of columns
dfs = dfs[['SCHOOL', 'TEACHERS', 'PUPIL ROLL', 'ASN PUPILS', 'ATTENDANCE RATE']]

# Add year
dfs['YEAR'] = '1819'

In [ ]:
# Read in data
dfs = pd.read_excel('./data/School+Contact+Details+2018.xlsx', 
                                sheet_name='Open at September 2018',
                                   skiprows=5,
                                       na_values='c')

# Tidy data
dfs = tidy(dfs)

# Remove (%) from colu\mn headings
dfs.columns = dfs.columns.str.replace('(%)', '')

# Create list of suffixes to remove
ls = ['(9)', '(8)', '(7)', '(6)', '(5)', '(4)', '(3)', '(2)', '(1)']

# Loop through suffix list
for i in ls:

    # Remove suffix
    dfs.columns = dfs.columns.str.removesuffix(i)
    
    # Strip white space
    dfs.columns = dfs.columns.str.strip()
    
# Fix duplicate column names
dfs.columns = ['SEEDCODE', 'LA NAME', 'CENTRE TYPE', 'SCHOOL NAME', 'ADDRESS 1',
       'ADDRESS 2', 'ADDRESS 3', 'POST CODE', 'E-MAIL', 'PHONE', 'PRIMARY',
       'SECONDARY', 'SPECIAL', 'PRIMARY ROLL', 'SECONDARY ROLL',
       'SPECIAL ROLL', 'PRIMARY FTE', 'SECONDARY FTE', 'SPECIAL FTE',
       '6-FOLD URBAN/RURAL MEASURE', '8-FOLD URBAN/RURAL MEASURE',
       'DENOMINATION', 'PROPORTION OF PUPILS FROM MINORITY ETHNIC GROUPS',
       'PROPORTION OF PUPILS WHO LIVE IN 20% MOST DEPRIVED DATAZONES IN SCOTLAND']

# Rename column headers
dfs = dfs.rename(columns={'SECONDARY FTE': 'TEACHERS'})
dfs = dfs.rename(columns={'SECONDARY ROLL': 'PUPIL ROLL'})
dfs = dfs.rename(columns={'SCHOOL NAME': 'SCHOOL'})

# Select Highland Secondary schools
dfs = dfs.loc[dfs['LA NAME'] == 'HIGHLAND']
dfs = dfs.loc[dfs['SECONDARY'] == 'SECONDARY']

# Add missing columns
dfs['ASN PUPILS'] = np.nan
dfs['ATTENDANCE RATE'] = np.nan

# Select subset of columns
dfs = dfs[['SCHOOL', 'TEACHERS', 'PUPIL ROLL', 'ASN PUPILS', 'ATTENDANCE RATE']]

# Add year
dfs['YEAR'] = '1718'

# Append df to list
frames.append(dfs)

In [ ]:
# Concat list into long df
adfs = pd.concat(frames, axis = 0, ignore_index = True)

In [ ]:
adfs.head()